In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from math import sqrt

from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Activation, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
#Run Local
#df = pd.read_csv("csv/book_info_complete.csv")
#Run on COLAB
df = pd.read_csv("/content/drive/My Drive/final_project/book_info_complete.csv")

In [4]:
df = df.dropna(axis=1)
df["Publication date"] = df["Publication date"].str[-4:].astype(int)
df = df.drop("Last updated", axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5816 entries, 0 to 5815
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             5816 non-null   object
 1   description       5816 non-null   object
 2   plot              5816 non-null   object
 3   csm_review        5816 non-null   object
 4   need_to_know      5816 non-null   object
 5   csm_rating        5816 non-null   int64 
 6   Genre             5816 non-null   object
 7   Book type         5816 non-null   object
 8   Publication date  5816 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 409.1+ KB


## Create the splits

In [ ]:
def splitter(df):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=999)
    for train_index, test_index in split.split(df, df['csm_rating']):
        train_data= df.loc[train_index]
        test_data = df.loc[test_index]
    
    
    return train_data, test_data

In [ ]:
train_data, test_data = splitter(df)

In [ ]:
x_tr, y_tr = train_data['need_to_know'].values, train_data["csm_rating"].values
x_val, y_val = test_data["need_to_know"].values, test_data["csm_rating"].values

In [8]:
print(x_tr.shape, y_tr.shape)

(4652,) (4652,)


In [9]:
print(x_val.shape, y_val.shape)

(1164,) (1164,)


## Prepare the Data

In [10]:
#Tokenize the sentences
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(x_tr))

#converting text into integer sequences
x_tr_seq  = tokenizer.texts_to_sequences(x_tr) 
x_val_seq = tokenizer.texts_to_sequences(x_val)

print(len(max(x_tr_seq, key=len)))
max_length = len(max(x_tr_seq, key=len))
print(len(min(x_tr_seq, key=len)))

332
10


In [11]:
#padding to prepare sequences of same length
x_tr_seq  = pad_sequences(x_tr_seq, maxlen=max_length)
x_val_seq = pad_sequences(x_val_seq, maxlen=max_length)
print(len(max(x_tr_seq, key=len)))
print(len(min(x_tr_seq, key=len)))

332
332


In [12]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

24351


## Create embeddings

In [13]:
word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))

Found 24350 unique tokens.


## Create the Model

In [14]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,input_length=max_length,trainable=True)) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='relu')) 

#Add loss function, metrics, optimizer
#optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(optimizer="RMSprop", loss='mean_squared_error', metrics=["mae"]) 

#Print summary of model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 332, 300)          7305300   
_________________________________________________________________
lstm (LSTM)                  (None, 332, 128)          219648    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 7,533,269
Trainable params: 7,533,269
Non-trainable params: 0
_________________________________________________________________
None


## Callbacks

In [ ]:
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_mae', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

## Fit the Model

In [16]:
history = model.fit(np.array(x_tr_seq),
                    np.array(y_tr),
                    batch_size=128,
                    epochs=1000,
                    validation_data=(np.array(x_val_seq),np.array(y_val)),
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/1000
37/37 [==============================] - 4s 121ms/step - loss: 23.6479 - mae: 3.9662 - val_loss: 15.1412 - val_mae: 3.2900 - lr: 0.0010
Epoch 2/1000
37/37 [==============================] - 4s 111ms/step - loss: 14.9511 - mae: 3.2647 - val_loss: 14.3005 - val_mae: 3.1952 - lr: 0.0010
Epoch 3/1000
37/37 [==============================] - 4s 106ms/step - loss: 13.2553 - mae: 3.0810 - val_loss: 11.6766 - val_mae: 2.8670 - lr: 0.0010
Epoch 4/1000
37/37 [==============================] - 4s 108ms/step - loss: 9.2788 - mae: 2.5530 - val_loss: 7.3630 - val_mae: 2.1968 - lr: 0.0010
Epoch 5/1000
37/37 [==============================] - 4s 108ms/step - loss: 5.4450 - mae: 1.8891 - val_loss: 4.5940 - val_mae: 1.6993 - lr: 0.0010
Epoch 6/1000
37/37 [==============================] - 4s 107ms/step - loss: 3.9306 - mae: 1.5637 - val_loss: 3.7002 - val_mae: 1.4841 - lr: 0.0010
Epoch 7/1000
37/37 [==============================] - 4s 109ms/step - loss: 3.1235 - mae: 1.3631 - val_loss: 3.3

In [17]:
#evaluation 
val_loss, val_mae = model.evaluate(x_val_seq, y_val)

print("The val_mae is %.3f." % val_mae)

37/37 [==============================] - 0s 11ms/step - loss: 3.2862 - mae: 1.4019
The val_mae is 1.402.


In [18]:
plt.plot(model.history.history["loss"], label="loss");
plt.plot(model.history.history["val_loss"], label="val_loss");
plt.legend();
plt.show();
plt.close();

plt.plot(model.history.history["mae"], label="mae");
plt.plot(model.history.history["val_mae"], label="val_mae");
plt.legend();

KeyError: ignored

## [Use Transfer Learning](https://www.analyticsvidhya.com/blog/2020/03/pretrained-word-embeddings-nlp/)

In [19]:
# load the whole embedding into memory
embeddings_index = dict()

with open("/content/drive/My Drive/Colab Notebooks/glove.6B/glove.6B.300d.txt") as f:

    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((size_of_vocabulary, 300))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [24]:
model2=Sequential()

#embedding layer
model2.add(Embedding(size_of_vocabulary,300,input_length=max_length,trainable=True)) 

#lstm layer
model2.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model2.add(GlobalMaxPooling1D())

#Dense Layer
model2.add(Dense(64,activation='relu')) 
model2.add(Dense(1,activation='relu')) 

#Add loss function, metrics, optimizer
#optimizer = tf.keras.optimizers.RMSprop(0.001)

model2.compile(optimizer="RMSprop", loss='mean_squared_error', metrics=["mae"]) 

#Print summary of model
print(model2.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 332, 300)          7305300   
_________________________________________________________________
lstm_2 (LSTM)                (None, 332, 128)          219648    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 7,533,269
Trainable params: 7,533,269
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
history = model2.fit(np.array(x_tr_seq),
                    np.array(y_tr),
                    batch_size=128,
                    epochs=1000,
                    validation_data=(np.array(x_val_seq),np.array(y_val)),
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/1000
37/37 [==============================] - 5s 122ms/step - loss: 23.2733 - mae: 3.9171 - val_loss: 14.9484 - val_mae: 3.2747 - lr: 0.0010
Epoch 2/1000
37/37 [==============================] - 4s 106ms/step - loss: 14.7230 - mae: 3.2505 - val_loss: 13.6234 - val_mae: 3.1266 - lr: 0.0010
Epoch 3/1000
37/37 [==============================] - 4s 107ms/step - loss: 10.3599 - mae: 2.7039 - val_loss: 5.6027 - val_mae: 1.9338 - lr: 0.0010
Epoch 4/1000
37/37 [==============================] - 4s 108ms/step - loss: 4.6331 - mae: 1.7053 - val_loss: 3.9897 - val_mae: 1.5779 - lr: 0.0010
Epoch 5/1000
37/37 [==============================] - 4s 107ms/step - loss: 3.4195 - mae: 1.4152 - val_loss: 4.4526 - val_mae: 1.6776 - lr: 0.0010
Epoch 6/1000
37/37 [==============================] - 4s 106ms/step - loss: 2.8124 - mae: 1.2659 - val_loss: 3.6089 - val_mae: 1.4563 - lr: 0.0010
Epoch 7/1000
37/37 [==============================] - 4s 107ms/step - loss: 2.1700 - mae: 1.1192 - val_loss: 3.24

In [26]:
#evaluation 
_, val_mae = model2.evaluate(x_val_seq, y_val)

print("The val_mae is %.3f." % val_mae)

37/37 [==============================] - 0s 12ms/step - loss: 3.7243 - mae: 1.4883
The val_mae is 1.488.


In [27]:
model2.save('/content/drive/My Drive/final_project/lstm_pntk')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/final_project/lstm_pntk/assets


In [28]:
plt.plot(model2.history.history["loss"], label="loss");
plt.plot(model2.history.history["val_loss"], label="val_loss");
plt.legend();
plt.show();
plt.close();

plt.plot(model2.history.history["mae"], label="mae");
plt.plot(model2.history.history["val_mae"], label="val_mae");
plt.legend();

KeyError: ignored

In [ ]:
a = model2.predict(x_val_seq)
b = model2.predict(x_val_seq)

csm_lstm_predictions_df = pd.DataFrame({"pntk_custom": list(a), "pntk_transfer": list(b)},
                              columns = ["pntk_custom", "pntk_transfer"], 
                              index=test_data.index)

csm_lstm_predictions_df.to_csv('/content/drive/My Drive/final_project/lstm_pntk.csv', index="False")
